In [2]:
from pymongo import MongoClient
mongo_url = 'mongodb://localhost:27017/'
client = MongoClient(mongo_url)
db = client.cloud_resources # database
inventory = db.inventory   # collection

In [2]:
#import pprint
#pprint.pprint(list(db.inventory.aggregate([match_stage, sort_stage, limit_stage, skip_stage], allowDiskUse=True)))

## $unwind pipeline operator 

``` works on array field only ```

### group by the context.application 

In [3]:
unwind_stage = {"$unwind": "$context.application"}
limit_stage = {"$limit": 1}

In [4]:
import pprint
pprint.pprint(list(db.inventory.aggregate([unwind_stage, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'),
  'context': {'application': 'cloud_based_app', 'environment': ['dev']},
  'location': {'availability_zone_code': ['ap-south-1a', 'ap-south-1b'],
               'region_code': 'ap-south-1'},
  'manage': 'enterprise_it',
  'provider_account': '12345678',
  'provider_account_name': 'test_account_1',
  'provider_code': 'TestProvider',
  'provider_resource_id': {'name': 'F0YC4G0483UW6VLIQD4WITJJT69A2VB962KIKC30XWXUOF7FLJXMPOAXADZHKDRLM',
                           'value': 'BPZQHZDCAURTJ7OJI2LQBS1MUA4SOWLQJHLV4GG56GRVR'},
  'provider_resource_name': 'Direct Connect - '
                            '8bdb002a-5abe-4228-ace9-7d9a2417d22f',
  'service_category_type': 'network',
  'status': 'Active',
  'type': 'service_instance'}]


```the application field inside context is converted into the string .
If the resource document has context.application ["app1", "app2", "app3"], this resource documents gets converted into 3 documents
and type of application field changed to string. 
``` 

``````Note```Only those documents will be passed to another stage , which has context.application field ```

## Thinking documents 

In [5]:
unwind_stage = {"$unwind": "$context.application"}
groupby_stage = {"$group": {"_id": "$provider_code", "sum": {"$sum": 1}}}

In [6]:
pprint.pprint(list(db.inventory.aggregate([unwind_stage, groupby_stage], allowDiskUse=True)))

[{'_id': 'aws', 'sum': 800000}, {'_id': 'TestProvider', 'sum': 200000}]


### $project pipeline operator 

#### Trim out the fields , which we dont want after each stage 

```To make efficient use memory and io,  we dont want to load whole document into all stages, hence we use $project to pass only selected fields into next stage.```

In [12]:
project_stage = {"$project": {"provider_code": 1, "provider_account": 1, "context.application": 1}}
limit_stage = {"$limit": 10}

In [13]:
pprint.pprint(list(db.inventory.aggregate([project_stage, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'),
  'context': {'application': ['cloud_based_app']},
  'provider_account': '12345678',
  'provider_code': 'TestProvider'},
 {'_id': ObjectId('5d02109af2e62711cbb03964'),
  'context': {'application': ['cloud_based_app']},
  'provider_account': '12345678',
  'provider_code': 'TestProvider'},
 {'_id': ObjectId('5d02109af2e62711cbb03965'),
  'context': {'application': ['cloud_based_app']},
  'provider_account': '12345678',
  'provider_code': 'TestProvider'},
 {'_id': ObjectId('5d02109af2e62711cbb03966'),
  'context': {'application': ['store']},
  'provider_account': '12345678',
  'provider_code': 'TestProvider'},
 {'_id': ObjectId('5d02109af2e62711cbb03967'),
  'context': {'application': ['d2ops']},
  'provider_account': '12345678',
  'provider_code': 'TestProvider'},
 {'_id': ObjectId('5d02109af2e62711cbb03968'),
  'context': {'application': ['cam']},
  'provider_account': '12345678',
  'provider_code': 'TestProvider'},
 {'_id': ObjectId('5d021

### Field rename by $project

In [14]:
project_stage = project_stage = {"$project": {"provider": "$provider_code", "account": "$provider_account", "resource_application": "$context.application"}}

In [15]:
limit_stage = {"$limit": 10}


In [16]:
pprint.pprint(list(db.inventory.aggregate([project_stage, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['cloud_based_app']},
 {'_id': ObjectId('5d02109af2e62711cbb03964'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['cloud_based_app']},
 {'_id': ObjectId('5d02109af2e62711cbb03965'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['cloud_based_app']},
 {'_id': ObjectId('5d02109af2e62711cbb03966'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['store']},
 {'_id': ObjectId('5d02109af2e62711cbb03967'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['d2ops']},
 {'_id': ObjectId('5d02109af2e62711cbb03968'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['cam']},
 {'_id': ObjectId('5d02109af2e62711cbb03969'),
  'account': '12345678',
  'provider': 'TestProvider',
  'resource_application': ['ca

## Reducing the number of fields in pipeline 
## Computation

In [24]:
project_stage = project_stage = {"$project": {"_id": 0, "type": 1, "provider_and_account": {"$concat": ["$provider_code", ' ', "$provider_account"]}, "resource_application": "$context.application"}}

In [25]:
limit_stage = {"$limit": 10}



In [26]:
pprint.pprint(list(db.inventory.aggregate([project_stage, limit_stage], allowDiskUse=True)))

[{'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['cloud_based_app'],
  'type': 'service_instance'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['cloud_based_app'],
  'type': 'service_instance'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['cloud_based_app'],
  'type': 'service_instance'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['store'],
  'type': 'independent_component'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['d2ops'],
  'type': 'independent_component'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['cam'],
  'type': 'independent_component'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['cam'],
  'type': 'service_instance'},
 {'provider_and_account': 'TestProvider 12345678',
  'resource_application': ['cam'],
  'type': 'service_instance'},
 {'provid

``` similary we can use $multiply $add, $substr (truncate string) ```

### $ifNull

In [28]:
project_stage_ifnull = {"$project": {"weight": {"$ifNull": ["$weight", 1.0]}}}

In [29]:
limit_stage = {"$limit": 10}

pprint.pprint(list(db.inventory.aggregate([project_stage_ifnull, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb03964'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb03965'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb03966'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb03967'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb03968'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb03969'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb0396a'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb0396b'), 'weight': 1.0},
 {'_id': ObjectId('5d02109af2e62711cbb0396c'), 'weight': 1.0}]


In [30]:
project_stage_ifnull = {"$project": {"context.project": {"$ifNull": ["$context.project", ["google"]]}}}
limit_stage = {"$limit": 10}

pprint.pprint(list(db.inventory.aggregate([project_stage_ifnull, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb03964'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb03965'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb03966'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb03967'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb03968'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb03969'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb0396a'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb0396b'),
  'context': {'project': ['google']}},
 {'_id': ObjectId('5d02109af2e62711cbb0396c'),
  'context': {'project': ['google']}}]


In [31]:
project_stage_ifnull = {"$project": {"context.application": {"$ifNull": ["$context.application", []]}}}
limit_stage = {"$limit": 10}

pprint.pprint(list(db.inventory.aggregate([project_stage_ifnull, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'),
  'context': {'application': ['cloud_based_app']}},
 {'_id': ObjectId('5d02109af2e62711cbb03964'),
  'context': {'application': ['cloud_based_app']}},
 {'_id': ObjectId('5d02109af2e62711cbb03965'),
  'context': {'application': ['cloud_based_app']}},
 {'_id': ObjectId('5d02109af2e62711cbb03966'),
  'context': {'application': ['store']}},
 {'_id': ObjectId('5d02109af2e62711cbb03967'),
  'context': {'application': ['d2ops']}},
 {'_id': ObjectId('5d02109af2e62711cbb03968'),
  'context': {'application': ['cam']}},
 {'_id': ObjectId('5d02109af2e62711cbb03969'),
  'context': {'application': ['cam']}},
 {'_id': ObjectId('5d02109af2e62711cbb0396a'),
  'context': {'application': ['cam']}},
 {'_id': ObjectId('5d02109af2e62711cbb0396b'),
  'context': {'application': ['cam']}},
 {'_id': ObjectId('5d02109af2e62711cbb0396c'),
  'context': {'application': ['cam']}}]


## Creating sub document using $project 

In [33]:
project_sub_doc_stage = {"$project": {"primary_info": {"code": "$provider_code", "details": {"account": "$provider_account", "status": "$status"}}}}

In [34]:
limit_stage = {"$limit": 10}

pprint.pprint(list(db.inventory.aggregate([project_sub_doc_stage, limit_stage], allowDiskUse=True)))

[{'_id': ObjectId('5d02109af2e62711cbb03963'),
  'primary_info': {'code': 'TestProvider',
                   'details': {'account': '12345678', 'status': 'Active'}}},
 {'_id': ObjectId('5d02109af2e62711cbb03964'),
  'primary_info': {'code': 'TestProvider',
                   'details': {'account': '12345678', 'status': 'Active'}}},
 {'_id': ObjectId('5d02109af2e62711cbb03965'),
  'primary_info': {'code': 'TestProvider',
                   'details': {'account': '12345678', 'status': 'Active'}}},
 {'_id': ObjectId('5d02109af2e62711cbb03966'),
  'primary_info': {'code': 'TestProvider',
                   'details': {'account': '12345678', 'status': 'Active'}}},
 {'_id': ObjectId('5d02109af2e62711cbb03967'),
  'primary_info': {'code': 'TestProvider',
                   'details': {'account': '12345678', 'status': 'Active'}}},
 {'_id': ObjectId('5d02109af2e62711cbb03968'),
  'primary_info': {'code': 'TestProvider',
                   'details': {'account': '12345678', 'status': 'Active'}}}

## Groupby Multiple keys

In [44]:
match_stage = {"$match": {"provider_code": "aws"}}
project_stage = {"$project": {"_id":0, "type": 1, "context": 1}}
unwind_stage = {"$unwind": "$context.application"}
sort_stage = {"$sort": {"context.application": 1}}
groupby_stage = {"$group": {"_id": {"app": "$context.application", "type": "$type"}, "sum": {"$sum": 1}}}

In [45]:
pprint.pprint(list(db.inventory.aggregate([match_stage, project_stage, unwind_stage, sort_stage, groupby_stage], allowDiskUse=True)))

[{'_id': {'app': 'store', 'type': 'independent_component'}, 'sum': 25415},
 {'_id': {'app': 'store', 'type': 'service_instance'}, 'sum': 89114},
 {'_id': {'app': 'finance_app', 'type': 'service_instance'}, 'sum': 89635},
 {'_id': {'app': 'finance_app', 'type': 'independent_component'}, 'sum': 25277},
 {'_id': {'app': 'aiops', 'type': 'independent_component'}, 'sum': 25544},
 {'_id': {'app': 'd2ops', 'type': 'service_instance'}, 'sum': 88760},
 {'_id': {'app': 'cam', 'type': 'independent_component'}, 'sum': 25431},
 {'_id': {'app': 'aiops', 'type': 'service_instance'}, 'sum': 88341},
 {'_id': {'app': 'cloud_based_app', 'type': 'independent_component'},
  'sum': 25196},
 {'_id': {'app': 'cam', 'type': 'service_instance'}, 'sum': 88268},
 {'_id': {'app': 'cloud_based_app', 'type': 'service_instance'}, 'sum': 88515},
 {'_id': {'app': 'd2ops', 'type': 'independent_component'}, 'sum': 25205},
 {'_id': {'app': 'ecommerce_app', 'type': 'independent_component'},
  'sum': 24943},
 {'_id': {'app'

In [46]:
match_stage = {"$match": {"provider_code": "aws"}}
project_stage = {"$project": {"_id":0, "type": 1, "context": 1}}
unwind_stage = {"$unwind": "$context.application"}
sort_stage = {"$sort": {"context.application": 1}}
groupby_stage = {"$group": {"_id": {"app": "$context.application", "type": "$type"}, "sum": {"$sum": 1}}}
cosmetic_output = {"$group": {"_id": "$_id.app", "data": {"$push": {"type": "$_id.type", "count": "$sum"}}}}

In [47]:
pprint.pprint(list(db.inventory.aggregate([match_stage, project_stage, unwind_stage, sort_stage, groupby_stage, cosmetic_output], allowDiskUse=True)))

[{'_id': 'ecommerce_app',
  'data': [{'count': 24943, 'type': 'independent_component'},
           {'count': 90356, 'type': 'service_instance'}]},
 {'_id': 'cam',
  'data': [{'count': 25431, 'type': 'independent_component'},
           {'count': 88268, 'type': 'service_instance'}]},
 {'_id': 'store',
  'data': [{'count': 25415, 'type': 'independent_component'},
           {'count': 89114, 'type': 'service_instance'}]},
 {'_id': 'finance_app',
  'data': [{'count': 89635, 'type': 'service_instance'},
           {'count': 25277, 'type': 'independent_component'}]},
 {'_id': 'aiops',
  'data': [{'count': 25544, 'type': 'independent_component'},
           {'count': 88341, 'type': 'service_instance'}]},
 {'_id': 'cloud_based_app',
  'data': [{'count': 25196, 'type': 'independent_component'},
           {'count': 88515, 'type': 'service_instance'}]},
 {'_id': 'd2ops',
  'data': [{'count': 88760, 'type': 'service_instance'},
           {'count': 25205, 'type': 'independent_component'}]}]


## renaming group _id field

In [50]:
match_stage = {"$match": {"provider_code": "aws"}}
project_stage = {"$project": {"_id":0, "type": 1, "context": 1}}
unwind_stage = {"$unwind": "$context.application"}
sort_stage = {"$sort": {"context.application": 1}}
groupby_stage = {"$group": {"_id": {"app": "$context.application", "type": "$type"}, "sum": {"$sum": 1}}}
cosmetic_output = {"$group": {"_id": "$_id.app", "data": {"$push": {"type": "$_id.type", "count": "$sum"}}}}
rename_id = {"$project": {"_id": 0, "application": "$_id", "data": 1}}

In [51]:
pprint.pprint(list(db.inventory.aggregate([match_stage, project_stage, unwind_stage, sort_stage, groupby_stage, cosmetic_output, rename_id], allowDiskUse=True)))

[{'application': 'ecommerce_app',
  'data': [{'count': 24943, 'type': 'independent_component'},
           {'count': 90356, 'type': 'service_instance'}]},
 {'application': 'cam',
  'data': [{'count': 25431, 'type': 'independent_component'},
           {'count': 88268, 'type': 'service_instance'}]},
 {'application': 'store',
  'data': [{'count': 25415, 'type': 'independent_component'},
           {'count': 89114, 'type': 'service_instance'}]},
 {'application': 'finance_app',
  'data': [{'count': 89635, 'type': 'service_instance'},
           {'count': 25277, 'type': 'independent_component'}]},
 {'application': 'aiops',
  'data': [{'count': 25544, 'type': 'independent_component'},
           {'count': 88341, 'type': 'service_instance'}]},
 {'application': 'cloud_based_app',
  'data': [{'count': 25196, 'type': 'independent_component'},
           {'count': 88515, 'type': 'service_instance'}]},
 {'application': 'd2ops',
  'data': [{'count': 88760, 'type': 'service_instance'},
           {'

In [58]:
match_stage = {"$match": {"provider_code": "aws"}}
project_stage = {"$project": {"_id":0, "type": 1, "context": 1}}
unwind_stage = {"$unwind": "$context.application"}
sort_stage = {"$sort": {"context.application": 1}}
groupby_stage = {"$group": {"_id": {"app": "$context.application", "type": "$type"}, "sum": {"$sum": 1}}}
cosmetic_output = {"$group": {"_id": "$_id.app", "data": {"$push": {"type": "$_id.type", "count": "$sum", "application": "$_id.app"}}}}
rename_id = {"$project": {"_id": 0, "data": 1}}

In [59]:
pprint.pprint(list(db.inventory.aggregate([match_stage, project_stage, unwind_stage, sort_stage, groupby_stage, cosmetic_output, rename_id], allowDiskUse=True)))

[{'data': [{'application': 'ecommerce_app',
            'count': 24943,
            'type': 'independent_component'},
           {'application': 'ecommerce_app',
            'count': 90356,
            'type': 'service_instance'}]},
 {'data': [{'application': 'cam',
            'count': 25431,
            'type': 'independent_component'},
           {'application': 'cam', 'count': 88268, 'type': 'service_instance'}]},
 {'data': [{'application': 'store',
            'count': 25415,
            'type': 'independent_component'},
           {'application': 'store',
            'count': 89114,
            'type': 'service_instance'}]},
 {'data': [{'application': 'finance_app',
            'count': 89635,
            'type': 'service_instance'},
           {'application': 'finance_app',
            'count': 25277,
            'type': 'independent_component'}]},
 {'data': [{'application': 'aiops',
            'count': 25544,
            'type': 'independent_component'},
           {'applicati